<a href="https://colab.research.google.com/github/VitalyGladyshev/LLM-engineering/blob/main/LLM_GLVV_HW08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 8 Векторы

## Гладышев ВВ

1. Взять набор документов (можно скачать историю сообщений тг канала) и построить на его основе
векторную базу данных. Выбрать любую FAISS, Qdrant, PGVector. По желанию можно самостоятельно
выбрать свою.
2. Проиндексировать с помощью любой модели SentenceTransformer. Можно брать из HF
(https://huggingface.co/spaces/mteb/leaderboard).
Обучение
Служба
поддержки
Mobile
3. Замерить как скорость работы меняется в зависимости от настроек индекса — сделать ключевые
выводы.


In [1]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 3.8 MB/s eta 0:00:00


In [2]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
pip install langchain-huggingface

In [7]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import math
import json

from time import sleep

import docx
from docx import Document
from docx.text.paragraph import Paragraph

from tqdm import tqdm

import time

random.seed(42)

### Парсинг текста

In [8]:
data = pd.DataFrame(columns=['header','text'])
data

,header,text


In [9]:
def save_to_dataframe(file_path):
    global data

    doc = Document(file_path)
    tab_cnt = 0
    curr_par_len = 0
    chapters = []

    lvl = 0
    number_arr = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    for el in doc.element.body:
        if el.tag.endswith('p'):

            paragraph = Paragraph(el, doc)
            style = paragraph.style.name

            if "Heading" in style:

                hd_parts = style.split()
                curr_lvl = int(hd_parts[-1]) - 1
                number_arr[curr_lvl] += 1

                # Проверим на заполнение номеров предыдущих уровней
                for level in range(curr_lvl):
                    if not number_arr[level]:
                        number_arr[level] = 1

                number = ""
                for i in range(curr_lvl+1):
                    if number:
                        number += "."
                    number += str(number_arr[i])

                while curr_lvl < lvl:
                    number_arr[lvl] = 0
                    lvl -= 1

                lvl = curr_lvl

                print(f"Добавлена глава: {number} {el.text.strip()}")
                # print("  "*curr_lvl + number + " " + el.text.strip())

                curr_par_len = 0

                header = el.text.strip()
                if type(header) == type(0.0):
                    header = ""
                chapters.append([number + " " + header, ""])

            elif paragraph.text:
                curr_par_len += len(paragraph.text)
                if len(chapters):
                    chapters[-1][-1] += paragraph.text
                else:
                    chapters.append([paragraph.text[:20], paragraph.text]) # текст до первого заголовка

        elif el.tag.endswith('tbl'):
            tab_cnt += 1

            table_data = []
            for row in doc.tables[tab_cnt-1].rows:
                row_data = [cell.text.strip() for cell in row.cells]
                table_data.append(row_data)

            if len(chapters):
                chapters[-1][-1] += str(table_data)
            else:
                chapters.append([str(table_data)[:20], str(table_data)])    # таблица до первого заголовка

    for chapter in chapters:
        new_row = {'header': chapter[0], 'text': chapter[1]}
        data = pd.concat([data,
                          pd.DataFrame([new_row], columns=data.columns)],
                         ignore_index = True)
    return data

In [10]:
data = save_to_dataframe("/content/merged_cr.docx")

Добавлена глава: 1 Назначение программного модуля CAD
Добавлена глава: 2 Условия выполнения программного модуля CAD
Добавлена глава: 3 Выполнение программного модуля CAD
Добавлена глава: 3.1 Запуск программного модуля CAD
Добавлена глава: 3.2 Управление приложениями
Добавлена глава: 3.3 Работа с документами
Добавлена глава: 3.3.1 Создание нового документа
Добавлена глава: 3.3.2 Открытие документов
Добавлена глава: 3.3.3 Сохранение документов
Добавлена глава: 3.3.4 Закрытие документа
Добавлена глава: 3.3.5 Настройка документов
Добавлена глава: 3.3.5.1 Свойства документа
Добавлена глава: 3.3.5.2 Атрибуты документа
Добавлена глава: 3.3.5.3 История документа
Добавлена глава: 3.3.5.4 Стили объектов
Добавлена глава: 3.4 Справочная система
Добавлена глава: 4 Интерфейс
Добавлена глава: 4.1 Окна программного модуля CAD
Добавлена глава: 4.1.1 Рабочие окна
Добавлена глава: 4.1.2 Вспомогательные окна
Добавлена глава: 4.2 Инструментальные панели
Добавлена глава: 4.2.1 Панель «Лента»
Добавлена глава

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   header  44 non-null     object
 1   text    44 non-null     object
dtypes: object(2)
memory usage: 836.0+ bytes


In [12]:
data.head()

,header,text
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...
2,3 Выполнение программного модуля CAD,
3,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...
4,3.2 Управление приложениями,Данное окно предназначено для осуществления уп...


In [13]:
data = data.loc[data.text != ""]
data.reset_index(drop=True, inplace=True)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   header  40 non-null     object
 1   text    40 non-null     object
dtypes: object(2)
memory usage: 772.0+ bytes


In [15]:
data.head()

,header,text
0,1 Назначение программного модуля CAD,Программный модуль CAD разработан на базе инте...
1,2 Условия выполнения программного модуля CAD,Достаточными условиями выполнения программного...
2,3.1 Запуск программного модуля CAD,Загрузка и запуск программного модуля CAD осущ...
3,3.2 Управление приложениями,Данное окно предназначено для осуществления уп...
4,3.3 Работа с документами,Перечень инструментов для работы с документами...


### Подготовка к сохранению в векторную базу

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [17]:
SIZE = 275

In [18]:
documents = []
metadatas = []

part_cnt = 0

for i in range(data.shape[0]):

    text_len = len(data.iloc[i]["text"])

    if data.iloc[i]["text"] == "-" or data.iloc[i]["text"] != data.iloc[i]["text"]:
        continue

    elif text_len <= SIZE:
        print(f"{i} длина: {len(data.iloc[i]['text'])}")
        documents.append(f"{data.iloc[i]['header'].strip()}\n{data.iloc[i]['text'].strip()}")
        metadatas.append({"header": data['header'][i],
                          "index": str(i),
                          "chunk":  "",
                          "content": data.iloc[i]["text"].strip()
                         })
        part_cnt += 1

    else:
        print(f"{i}")
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=SIZE,
            chunk_overlap=0,
            separators=["\n", " ", ""]  # "\n\n", Порядок приоритета разделителей
        )

        chunks = text_splitter.split_text(data.iloc[i]["text"].strip())

        len_ch = len(chunks)
        step = 13
        if len_ch > step:
            for j in range(step, len_ch, step):
                part = " ".join(chunks[j-(step):j])
                for k in range(j - step, j):
                    documents.append(f"{data['header'][i].strip()}\n{chunks[k].strip()}")
                    metadatas.append({"header": data['header'][i],
                                      "index": str(i),
                                      "chunk":  chunks[k],
                                      "content": part.strip()
                                     })
                    print(f"part len: {len(part)} chunk len: {len(chunks[k])}") # {chunks[k]:50}")
                part_cnt += 1
        else:
            j = 0

        if j < len_ch or len_ch < step:
            part = " ".join(chunks[j:])
            for k in range(j, len_ch):
                documents.append(f"{data['header'][i].strip()}\n{chunks[k].strip()}")
                metadatas.append({"header": data['header'][i],
                                  "index": str(i),
                                  "chunk":  chunks[k],
                                  "content": part.strip()
                                 })
                print(f"part len: {len(part)} chunk len: {len(chunks[k])}") # {chunks[k]:50}")
            part_cnt += 1

0
part len: 598 chunk len: 263
part len: 598 chunk len: 272
part len: 598 chunk len: 61
1
part len: 347 chunk len: 273
part len: 347 chunk len: 73
2
part len: 825 chunk len: 267
part len: 825 chunk len: 266
part len: 825 chunk len: 265
part len: 825 chunk len: 24
3
part len: 1297 chunk len: 271
part len: 1297 chunk len: 257
part len: 1297 chunk len: 269
part len: 1297 chunk len: 265
part len: 1297 chunk len: 231
4 длина: 73
5
part len: 1068 chunk len: 265
part len: 1068 chunk len: 271
part len: 1068 chunk len: 247
part len: 1068 chunk len: 270
part len: 1068 chunk len: 11
6
part len: 2004 chunk len: 275
part len: 2004 chunk len: 272
part len: 2004 chunk len: 267
part len: 2004 chunk len: 248
part len: 2004 chunk len: 269
part len: 2004 chunk len: 273
part len: 2004 chunk len: 258
part len: 2004 chunk len: 135
7
part len: 1177 chunk len: 270
part len: 1177 chunk len: 269
part len: 1177 chunk len: 261
part len: 1177 chunk len: 270
part len: 1177 chunk len: 103
8 длина: 65
9
part len: 127

### Сохранение в векторную базу

In [41]:
%pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 10.5 MB/s eta

In [22]:
import torch
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from chromadb import PersistentClient

from chromadb.utils import embedding_functions
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from sentence_transformers import SentenceTransformer

In [34]:
import chromadb
from uuid import uuid4

In [23]:
documents[:5]

['1 Назначение программного модуля CAD\nПрограммный модуль CAD разработан на базе интегрированной инженерной программной платформы и программно-математического ядра трехмерного моделирования «RGK».Функциональное назначение программного модуля CAD:создание и редактирование геометрической модели объектов',
 '1 Назначение программного модуля CAD\nпроектирования, включая 3D- и 2D-представления;задание атрибутивной информации, связанной с точностными, размерными, технологическими параметрами объектов проектирования (PMI);оформление графической документации в соответствии с требованиями стандартов (ЕСКД);визуализация',
 '1 Назначение программного модуля CAD\nобъектов проектирования с различными параметрами отображения.',
 '2 Условия выполнения программного модуля CAD\nДостаточными условиями выполнения программного модуля CAD являются системные требования:ОС – Microsoft Windows версии не ниже 10, Astra Linux версии 1.7 и выше;процессор Intel Core i3 или выше;видеокарта NVIDIA или AMD с память

In [24]:
metadatas[:3]

[{'header': '1 Назначение программного модуля CAD',
  'index': '0',
  'chunk': 'Программный модуль CAD разработан на базе интегрированной инженерной программной платформы и программно-математического ядра трехмерного моделирования «RGK».Функциональное назначение программного модуля CAD:создание и редактирование геометрической модели объектов',
  'content': 'Программный модуль CAD разработан на базе интегрированной инженерной программной платформы и программно-математического ядра трехмерного моделирования «RGK».Функциональное назначение программного модуля CAD:создание и редактирование геометрической модели объектов проектирования, включая 3D- и 2D-представления;задание атрибутивной информации, связанной с точностными, размерными, технологическими параметрами объектов проектирования (PMI);оформление графической документации в соответствии с требованиями стандартов (ЕСКД);визуализация объектов проектирования с различными параметрами отображения.'},
 {'header': '1 Назначение программного

In [25]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="intfloat/multilingual-e5-large")
sentence_transformer_ef_cl = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [28]:
persistent_client_serv = chromadb.PersistentClient(path="./db_data")
collection_serv = persistent_client_serv.get_or_create_collection("data_collection", embedding_function=sentence_transformer_ef)

In [31]:
uuids = [str(uuid4()) for _ in range(len(documents))]

In [32]:
start = time.time()

collection_serv.add(
    ids=uuids,
    documents=documents,
    metadatas=metadatas
)

end = time.time()
print(f"Время исполнения: {(end - start):.3f} с.")

Время исполнения: 258.291 с.


In [47]:
start_time = time.time()

vector_store_serv_from_client = Chroma(
    client=persistent_client_serv,
    collection_name="data_collection",
    embedding_function=sentence_transformer_ef_cl,
)

response_time = time.time() - start_time
print(f"Время построения клиента: {response_time:.3f} секунды")

Время построения клиента: 0.015 секунды


In [48]:
question = "Программный модуль"

In [49]:
vect_res = vector_store_serv_from_client.similarity_search(
    question,
    k=5
)

In [50]:
len(vect_res)

5

In [52]:
vect_res[0]

Document(id='836faab4-9a8d-4601-b0a3-5f10fc4742fd', metadata={'chunk': 'программного модуля CAD.', 'content': 'Загрузка и запуск программного модуля CAD осуществляется из приложенияоболочки пользовательского интерфейса программного модуля «Технологическая платформа» RPLM-Application.В случае успешного запуска программы в главном окне технологической платформы будет отображена стартовая страница «Приветствие»\xa0(рис.\xa0#).Стартовая страница «Приветствие»Рисунок\xa0Данная страница содержит несколько блоков:«Стартовая страница» – содержит кнопки запуска установок интерфейса и вызова справки;«Избранное» – основные инструменты модулей информационной подсистемы;«Создать документ» – набор прототипов для создания нового документа; «Недавние файлы» – перечень последних открываемых документов, а также кнопки открытия документа и выбора каталога.Страницу «Приветствие» также можно открыть из выпадающего меню настроек программного модуля CAD.', 'index': '2', 'header': '3.1 Запуск программного мод

In [54]:
vect_res[0].metadata

{'chunk': 'программного модуля CAD.',
 'content': 'Загрузка и запуск программного модуля CAD осуществляется из приложенияоболочки пользовательского интерфейса программного модуля «Технологическая платформа» RPLM-Application.В случае успешного запуска программы в главном окне технологической платформы будет отображена стартовая страница «Приветствие»\xa0(рис.\xa0#).Стартовая страница «Приветствие»Рисунок\xa0Данная страница содержит несколько блоков:«Стартовая страница» – содержит кнопки запуска установок интерфейса и вызова справки;«Избранное» – основные инструменты модулей информационной подсистемы;«Создать документ» – набор прототипов для создания нового документа; «Недавние файлы» – перечень последних открываемых документов, а также кнопки открытия документа и выбора каталога.Страницу «Приветствие» также можно открыть из выпадающего меню настроек программного модуля CAD.',
 'index': '2',
 'header': '3.1 Запуск программного модуля CAD'}

In [55]:
vect_res[0].page_content

'3.1 Запуск программного модуля CAD\nпрограммного модуля CAD.'

In [58]:
for i in range(len(vect_res)):
    print(f"{vect_res[i].page_content}\n")

3.1 Запуск программного модуля CAD
программного модуля CAD.

1 Назначение программного модуля CAD
объектов проектирования с различными параметрами отображения.

1 Назначение программного модуля CAD
Программный модуль CAD разработан на базе интегрированной инженерной программной платформы и программно-математического ядра трехмерного моделирования «RGK».Функциональное назначение программного модуля CAD:создание и редактирование геометрической модели объектов

3.2 Управление приложениями
помощью кнопки «Удалить».Изменение состояния модулей происходит после перезагрузки системы.Для автоматического запуска программного компонента вместе с системой, необходимо установить флаг в графе «Автозапуск» напротив необходимого модуля.Закрытие окна «Приложения»

4.1 Окна программного модуля CAD
закрытие любого из главных окон не меняет функциональность работы программного модуля CAD в текущем сеансе. Программный модуль CAD завершает работу при закрытии последнего главного окна.Открытие нового главног

In [65]:
docs = []
for i in range(len(vect_res)):
    if vect_res[i].metadata["content"] not in docs:
        docs.append(vect_res[i].metadata["content"])
        print(f"{i+1} {vect_res[i].metadata["content"]}\n")

1 Загрузка и запуск программного модуля CAD осуществляется из приложенияоболочки пользовательского интерфейса программного модуля «Технологическая платформа» RPLM-Application.В случае успешного запуска программы в главном окне технологической платформы будет отображена стартовая страница «Приветствие» (рис. #).Стартовая страница «Приветствие»Рисунок Данная страница содержит несколько блоков:«Стартовая страница» – содержит кнопки запуска установок интерфейса и вызова справки;«Избранное» – основные инструменты модулей информационной подсистемы;«Создать документ» – набор прототипов для создания нового документа; «Недавние файлы» – перечень последних открываемых документов, а также кнопки открытия документа и выбора каталога.Страницу «Приветствие» также можно открыть из выпадающего меню настроек программного модуля CAD.

2 Программный модуль CAD разработан на базе интегрированной инженерной программной платформы и программно-математического ядра трехмерного моделирования «RGK».Функциональн

## Выводы

Длительность заполнения векторной базы зависит:
- от разбиения (количества чанков)
- от модели эмбеддингов
- от разрядности эмбеддингов
- от размещения модели на GPU